In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

**import library**

In [2]:
import numpy as np 
from numpy.random import seed
import pandas as pd 
from matplotlib import pyplot as plt
# DNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics


**讀取檔案**

In [4]:
train = pd.read_csv("train.csv", parse_dates=['date'], infer_datetime_format=True, dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'int32',
    }, usecols=['store_nbr', 'family', 'date', 'sales','onpromotion'])


test = pd.read_csv("test.csv", parse_dates=['date'], infer_datetime_format=True, dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'int32',
    })

oil = pd.read_csv("oil.csv", parse_dates=['date'], infer_datetime_format=True, dtype={
        'dcoilwtico':'float32'
    })

holiday = pd.read_csv("holidays_events.csv", parse_dates=['date'], infer_datetime_format=True, dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool'
    })



In [5]:
train['date'] = train.date.dt.to_period('D')
test['date'] = test.date.dt.to_period('D')


In [6]:
train=train.set_index(['store_nbr', 'family', 'date']).sort_index()
test=test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [7]:
start_date = '2017-04-01'
end_date = '2017-08-15'

In [8]:
y = train.drop(['onpromotion'], axis=1).unstack(['store_nbr', 'family']).loc[start_date:end_date,'sales']
y.shape

(137, 1782)

In [9]:
y.head()

store_nbr           1                                                \
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-04-01        9.0       0.0    1.0    3229.0   0.0   526.249023   
2017-04-02        4.0       0.0    1.0    1210.0   0.0   180.339005   
2017-04-03       11.0       0.0    2.0    2097.0   0.0   444.856995   
2017-04-04        3.0       0.0    4.0    2249.0   1.0   403.819000   
2017-04-05        5.0       0.0    1.0    2687.0   2.0   499.385010   

store_nbr                                            ...         9  \
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-04-01        14.0    858.0  1151.0  243.272003  ...       3.0   
2017-04-02         3.0    281.0   446.0   86.641998  ...       4.0   
2017-04-03        11.0    801.0   794.0  166.119995  ...       3.0   
2017-04-04        19.0    673.0   725.0  149.078003  ...       0.0   
2017-04-05        18.0   1057.0  1074.0  216.742996  ...       1.0   

store_nbr                                                                  \
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-04-01  671.481018        1524.0         20.0                    24.0   
2017-04-02  513.000977         842.0         19.0                    29.0   
2017-04-03  481.912018         742.0          7.0                    14.0   
2017-04-04  290.717010         594.0          4.0                    15.0   
2017-04-05  410.497009         633.0          2.0                    20.0   

store_nbr                                                                      \
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-04-01  851.388000     311.147003  2625.523926                        1.0   
2017-04-02  621.062012     229.181000  2303.476074                        3.0   
2017-04-03  532.640991     132.583008  1697.151001                        2.0   
2017-04-04  451.968994     127.585999  2506.139893                        2.0   
2017-04-05  395.341003      84.173996  1349.001953                        1.0   

store_nbr              
family        SEAFOOD  
date                   
2017-04-01  40.446999  
2017-04-02  15.000000  
2017-04-03  16.000000  
2017-04-04  15.000000  
2017-04-05  13.373000  

[5 rows x 1782 columns]

In [10]:
yvalue=y.values
yvalue

array([[9.000000e+00, 0.000000e+00, 1.000000e+00, ..., 2.625524e+03,
        1.000000e+00, 4.044700e+01],
       [4.000000e+00, 0.000000e+00, 1.000000e+00, ..., 2.303476e+03,
        3.000000e+00, 1.500000e+01],
       [1.100000e+01, 0.000000e+00, 2.000000e+00, ..., 1.697151e+03,
        2.000000e+00, 1.600000e+01],
       ...,
       [1.000000e+00, 0.000000e+00, 1.000000e+00, ..., 1.693607e+03,
        2.000000e+02, 2.000000e+01],
       [1.000000e+00, 0.000000e+00, 6.000000e+00, ..., 1.348425e+03,
        1.820000e+02, 1.700000e+01],
       [4.000000e+00, 0.000000e+00, 4.000000e+00, ..., 2.419729e+03,
        1.210000e+02, 1.600000e+01]], dtype=float32)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # 將資料標準化到 0-1

In [6]:
look_back = 10 # 往前看10天

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 4)                 240       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


In [16]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)


In [17]:

final_ans = []

for i in range(1782): # 遍歷每個店家的每個商品
  print("i=",i)

  yvaluetrain = scaler.fit_transform(yvalue[:,i:i+1]) # 每個商品所有日期的價格，進行標準化

  # train_size = int(len(yvaluetrain) * 0.67)
  # test_size = len(yvaluetrain) - train_size
  # train, test = yvaluetrain[0:train_size,:], yvaluetrain[train_size:len(yvaluetrain),:]

  trainX, trainY = create_dataset(yvaluetrain, look_back) # 訓練資料 訓練標籤
  # testX, testY = create_dataset(test, look_back)

  trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1])) # (訓練筆數 , feature數量 , lookback天數)
  # testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

  model.fit(trainX, trainY, epochs=1, batch_size=1,verbose=2)


  new_x = []
  tx = yvaluetrain[len(yvaluetrain)-look_back-1:len(yvaluetrain)-1,0] # 取train最後10天來進行預測
  new_x.append(tx)
  new_x = np.array(new_x)
  new_x = np.reshape(new_x, (new_x.shape[0], 1, new_x.shape[1]))


  for j in range(16):
    y_pred_one = model.predict(new_x[:,:,j:])
    y_pred_one = y_pred_one.reshape(1,1,1)
    new_x = np.concatenate([new_x,y_pred_one],axis=2) # 將新預測出來的結果當作下一天的預測資料

  ans_y = new_x[:,:,len(new_x)-17:].reshape(16,1)
  ans_y = scaler.inverse_transform(ans_y) # 將標準化後的資料復原

  final_ans.append(ans_y)




i= 0
126/126 - 1s - loss: 0.0402
i= 1
126/126 - 0s - loss: 0.0055
i= 2
126/126 - 0s - loss: 0.0482
i= 3
126/126 - 0s - loss: 0.0572
i= 4
126/126 - 0s - loss: 0.0511
i= 5
126/126 - 0s - loss: 0.0551
i= 6
126/126 - 0s - loss: 0.0437
i= 7
126/126 - 0s - loss: 0.0479
i= 8
126/126 - 0s - loss: 0.0376
i= 9
126/126 - 0s - loss: 0.0341
i= 10
126/126 - 0s - loss: 0.0298
i= 11
126/126 - 0s - loss: 0.0338
i= 12
126/126 - 0s - loss: 0.0317
i= 13
126/126 - 0s - loss: 0.0323
i= 14
126/126 - 0s - loss: 0.0324
i= 15
126/126 - 0s - loss: 0.0472
i= 16
126/126 - 0s - loss: 0.0438
i= 17
126/126 - 0s - loss: 0.0419
i= 18
126/126 - 0s - loss: 0.0403
i= 19
126/126 - 0s - loss: 0.0469
i= 20
126/126 - 0s - loss: 0.0161
i= 21
126/126 - 0s - loss: 0.0630
i= 22
126/126 - 0s - loss: 0.0287
i= 23
126/126 - 0s - loss: 0.0324
i= 24
126/126 - 0s - loss: 0.0301
i= 25
126/126 - 0s - loss: 0.0302
i= 26
126/126 - 0s - loss: 0.0276
i= 27
126/126 - 0s - loss: 0.0378
i= 28
126/126 - 0s - loss: 0.0303
i= 29
126/126 - 0s - los

In [ ]:
final_ans = np.reshape(final_ans,(1782,16))
final_ans_transpose = final_ans.transpose()
final_ans_transpose
  

In [ ]:
date_index=[
  '2017-08-16',
  '2017-08-17',
  '2017-08-18',
  '2017-08-19',
  '2017-08-20',
  '2017-08-21',
  '2017-08-22',
  '2017-08-23',
  '2017-08-24',
  '2017-08-25',
  '2017-08-26',
  '2017-08-27',
  '2017-08-28',
  '2017-08-29',
  '2017-08-30',
  '2017-08-31'
]

In [ ]:
y_submit = pd.DataFrame(final_ans_transpose.clip(0.0),index=date_index, columns=y.columns)

In [ ]:
y_submit.head(16)

In [ ]:
y_submit = (y_submit
      .stack(['store_nbr', 'family'])
      .reset_index()
      .rename(columns={0:"sales", 'level_0':'date'})
      .set_index(['date', 'store_nbr', 'family'])
      .sort_index())

In [ ]:
y_submit = y_submit.join(test.id).reindex(columns=['id', 'sales'])

In [ ]:
for i in range(28512):
  y_submit.iloc[i,y_submit.columns.get_loc('id')]=3000888+i

In [ ]:
y_submit=y_submit.astype({"id":int})

In [ ]:
y_submit.head() , y_submit.tail()

In [ ]:
y_submit.to_csv('submission.csv', index=False)

# 測試 (用來畫圖交作業)

In [ ]:
start_date = '2017-04-01'
end_date = '2017-08-15'

y = train.drop(['onpromotion'], axis=1).unstack(['store_nbr', 'family']).loc[start_date:end_date,'sales']
yvalue=y.values

In [ ]:
look_back = 30 # 往前看10天

model = Sequential()
model.add(LSTM(8, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='', optimizer='adam')

In [ ]:
yvaluetrain = scaler.fit_transform(yvalue[:,0:1])

look_back=30

train_size = int(len(yvaluetrain) * 0.67)
test_size = len(yvaluetrain) - train_size
train, test = yvaluetrain[0:train_size,:], yvaluetrain[train_size:len(yvaluetrain),:]

trainX, trainY = create_dataset(yvaluetrain, look_back) # 訓練資料 訓練標籤
testX, testY = create_dataset(test, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1])) # (訓練筆數 , feature數量 , lookback天數)
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# 回復預測資料值為原始數據的規模
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainY = trainY.reshape(106,)

In [ ]:
plt.figure(figsize=(24,8))
plt.plot(trainPredict,label='train_predict')
plt.plot(trainY,label='train_Y')
plt.legend()
plt.show()

In [ ]:
testY = testY.reshape(15,)

In [ ]:
plt.figure(figsize=(24,8))
plt.plot(testPredict,label='test_predict')
plt.plot(testY,label='test_Y')
plt.legend()
plt.show()